In [1]:
import pandas as pd 
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, MaxPooling2D, BatchNormalization, Dropout
import numpy as np
import cv2 
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt

In [2]:
image_size = (208,208)
batch_size = 32

In [3]:
def _сorrecting_data():
    x = []
    y = []
    boxes = []
    df = pd.read_csv('_annotations.csv') 
    for x1,y1,x2,y2 in zip(df['xmin'].apply(lambda x: x // 2), list(df['ymin'].apply(lambda x: x // 2)), 
                       list(df['xmax'].apply(lambda x: x // 2)), list(df['ymax'].apply(lambda x: x // 2))):
        arr = [x1, y1, x2, y2]
        boxes.append(arr)
    return df, boxes

In [4]:
def data_metrics(df):
    images = []
    
    for index, i in df.iterrows():
        image = cv2.imread(index)
        images.append(image)
    return images

In [5]:
def _data(df):
    x = []
    y = []
    boxes = []
    for index, g in df.iterrows():
        if (df['height'][index] !=  0) and (df['width'][index] !=  0):
            break
        else:
            boxes.append([None,None,None,None])
    for x1,y1,x2,y2 in zip(df['xmin'].apply(lambda x: x // 2), list(df['ymin'].apply(lambda x: x // 2)), 
                   list(df['xmax'].apply(lambda x: x // 2)), list(df['ymax'].apply(lambda x: x // 2))):
        arr = [x1, y1, x2, y2]
        boxes.append(arr)
    return boxes

In [6]:
def __data(df):
    x = []
    y = []
    boxes = []
#     for index, g in df.iterrows():
#         if (df['height'][index] !=  0) and (df['width'][index] !=  0):
#             break
#         else:
#             boxes.append([0,0,0,0])
    for height, width, x1, y1, x2, y2 in zip(df['height'], df['width'], df['xmin'].apply(lambda x: x // 2), list(df['ymin'].apply(lambda x: x // 2)), 
                   list(df['xmax'].apply(lambda x: x // 2)), list(df['ymax'].apply(lambda x: x // 2))):
        if height == 0 and width ==0:
            boxes.append([-1,-1,-1,-1])
        else:
            arr = [int(x1), int(y1), int(x2), int(y2)]
            boxes.append(arr)
    return boxes

In [7]:
def data_test_fun(images, coordinates, image_size):
    scaled_regions = []
    labels = []
    for image, coords in zip(images, coordinates):
        x, y, w, h = coords
        if x + y + w + h != -4:
        #if x is not None and y is not None and w is not None and h is not None:
            scaled_region = cv2.resize(image, image_size)
            scaled_regions.append(scaled_region)
            labels.append(1)  # Предполагая, что все извлеченные области - это машины
            # Преобразование входных данных в массивы NumPy
        else:
            #image = np.array(image)
            scaled_regions.append(cv2.resize(image, image_size))
            labels.append(0)  # Предполагая, что все извлеченные области - это не машины
    return np.array(scaled_regions), np.array(coordinates), np.array(labels)

In [8]:
df_trash = pd.DataFrame({'filename': os.listdir('DataSet_cars_valid\\1_trash')})
df_trash = df_trash.set_index('filename')
df_trash = df_trash.astype(str)

In [9]:
%cd DataSet_cars_valid/valid_test
df_train, boxes_train = _сorrecting_data()
df_train = df_train.set_index('filename')
train = pd.concat([df_train, df_trash])
train = train.fillna(0)
train = train.sort_values(by = 'filename')
boxes_train = __data(train)
image_train = data_metrics(train)
%cd ../..

C:\Users\Matvey\Проект\Project\DataSet_cars_valid\valid_test
C:\Users\Matvey\Проект\Project


In [167]:
# %cd DataSet_cars_valid/valid
# train, boxes_train = _сorrecting_data()
# train = train.set_index('filename')
# #train = pd.concat([df_train, df_trash])
# #train = train.fillna(0)
# #train = train.sort_values(by = 'filename')
# boxes_train = __data(train)
# image_train = data_metrics(train)
# %cd ../..

C:\Users\Matvey\Проект\Project\DataSet_cars_valid\valid
C:\Users\Matvey\Проект\Project


In [10]:
# boxes_train = np.array(boxes_train)
# boxes_train = boxes_train.astype(np.int64)
# boxes_train = boxes_train.tolist()

region_train, image_coodrs,image_class  = data_test_fun(image_train, boxes_train, image_size)

In [38]:
len(image_coodrs)

5139

In [48]:
for i, image in enumerate(region_train):
    x, y, width, height = image_coodrs[i]
    if x+y+width+height != -4:
    #if x is not None and y is not None and width is not None and height is not None:
#         cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)
#         cv2.imshow('Image', image)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        continue
    else: 
        cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.imshow('Image', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

KeyboardInterrupt: 

In [11]:
train_data, val_data, train_labels, val_labels = train_test_split(
    region_train, image_coodrs, test_size=0.2, random_state=42)

In [191]:
train_labels

array([[1, 5, 103, 98],
       [19, 9, 99, 52],
       [12, 10, 96, 97],
       ...,
       [27, 37, 74, 72],
       [3, 11, 100, 103],
       [6, 22, 99, 86]], dtype=object)

In [12]:
model = keras.Sequential([
 Conv2D(128, (3, 3), activation='relu', input_shape=(208, 208, 3)),  # свёрточный слой
    #попробуй по 2-3 свёрточных слоя сделать
    MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

    Conv2D(128, (3, 3), activation='relu'),  # свёрточный слой
    MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

    Conv2D(256, (3, 3), activation='relu'),  # свёрточный слой
    MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

    Conv2D(256, (3, 3), activation='relu'),  # свёрточный слой
    MaxPooling2D((2, 2), strides=2),  # уменьшеи карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

    Conv2D(512, (3, 3), activation='relu'),  # свёрточный слой
    MaxPooling2D((2, 2), strides=2),  # уменьшение карт признаков
    Dropout(0.2),  # отключение 20 процентов нейронов

    Flatten(),
    Dense(512, activation='relu'),  # обычные нейроны
    Dropout(0.2),  # отключение 20 процентов нейронов
    Dense(256, activation='relu'),  # обычные нейроны
    Dropout(0.2),  # отключение 20 процентов нейронов
    Dense(128, activation='relu'),  # обычные нейроны
    Dropout(0.2),  # отключение 20 процентов нейронов
    Dense(4, activation='linear'),  # выходной слой
])

In [13]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [14]:
train_data, val_data, train_labels, val_labels = train_test_split(
    region_train, image_coodrs, test_size=0.2, random_state=42)

In [17]:
history = model.fit(train_data, train_labels, epochs = 50, batch_size = batch_size, validation_data = (val_data, val_labels))

Epoch 1/50
129/129 [==============================] - 23s 177ms/step - loss: 520.0608 - mse: 520.0608 - val_loss: 1800.6527 - val_mse: 1800.6527
Epoch 2/50
129/129 [==============================] - 19s 146ms/step - loss: 534.3242 - mse: 534.3242 - val_loss: 1594.1605 - val_mse: 1594.1605
Epoch 3/50
129/129 [==============================] - 18s 136ms/step - loss: 650.1933 - mse: 650.1933 - val_loss: 1773.0840 - val_mse: 1773.0840
Epoch 4/50
129/129 [==============================] - 27s 211ms/step - loss: 579.7346 - mse: 579.7346 - val_loss: 1746.5499 - val_mse: 1746.5499
Epoch 5/50
129/129 [==============================] - 19s 148ms/step - loss: 570.1947 - mse: 570.1947 - val_loss: 1403.5217 - val_mse: 1403.5217
Epoch 6/50
129/129 [==============================] - 20s 159ms/step - loss: 535.6735 - mse: 535.6735 - val_loss: 2186.8003 - val_mse: 2186.8003
Epoch 7/50
129/129 [==============================] - 18s 137ms/step - loss: 523.1392 - mse: 523.1392 - val_loss: 1932.5085 - val_

In [18]:
model.save('metrics_.h5')